# Introduction

One of the main objectives participants of fantasy sports competitions try to achieve is assembling the best-performing team over the course of an entire season. Typically, performance is based on points accrued by each participant's team chosen during a each period of the competition, which can last days, weeks, or months. Most major sports leagues have branded fantasy leagues for theire respective competitions. In this study, data collected from the English Premier League's Fantasy Premier League over the past five years will be disseminated to understand the challenge of choosing a team of 11 starters and four substitues can be, such that the points accrued by the team over each competition period is maximized.

The rules for playing FPL can be found using the following URL - https://fantasy.premierleague.com/help/rules. Rules will be mentioned as needed as this analysis moves towards a comprehensive description of what factors must be considered when assembling a team for each gameweek.

# 1 - Exploring the Data

First, we'll import some important libraries for doing basic exploratory analysis to describe the data.

In [1]:
import pandas as pd

# save filepath to variable for easier access
fpl_data_file_path = '../data/FPL17-GW0.csv'

# read the data and store data in DataFrame titled fpl_data
fpl_data = pd.read_csv(fpl_data_file_path) 

# print a summary of the data in FPL data
fpl_data.describe()

,Cost,PointsLastRound,TotalPoints,AveragePoints,AveragePointsPerDollar,TotalPointsPerDollar,GameweekWeighting,TransfersOut,YellowCards,GoalsConceded,...,GW30Forecast,GW31Forecast,GW32Forecast,GW33Forecast,GW34Forecast,GW35Forecast,GW36Forecast,GW37Forecast,GW38Forecast,ICTIndex
count,5.010000e+02,501.0,501.000000,5.010000e+02,5.010000e+02,501.000000,501.0,501.0,501.000000,501.000000,...,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000
mean,5.427146e+06,0.0,53.690619,1.031201e+00,1.727529e-07,0.000009,0.0,0.0,2.245509,18.546906,...,0.991617,1.018762,0.961078,0.979441,0.988822,1.006387,0.994212,1.028343,1.012375,70.408383
std,1.325587e+06,0.0,52.914706,1.483116e+00,2.341806e-07,0.000008,0.0,0.0,2.796713,18.125129,...,1.651790,1.710254,1.578690,1.640609,1.615424,1.685372,1.638654,1.685288,1.690919,79.389004
min,4.000000e+06,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.500000e+06,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000e+06,0.0,45.000000,5.135877e-27,9.337958e-34,0.000008,0.0,0.0,1.000000,15.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000
75%,5.500000e+06,0.0,90.000000,2.105263e+00,4.052632e-07,0.000016,0.0,0.0,4.000000,31.000000,...,1.700000,1.600000,1.700000,1.600000,1.800000,1.600000,1.800000,1.800000,1.700000,111.300000
max,1.250000e+07,0.0,264.000000,6.947368e+00,7.842105e-07,0.000030,0.0,0.0,14.000000,70.000000,...,9.900000,8.500000,9.900000,9.400000,8.600000,8.400000,8.400000,9.900000,8.600000,453.400000


Viewing these summary statistics show us some unusual figures, such as the PointsLastRound column being 0.0 for every row. That's because we are using data representing available player data for 2017-2018 Fantasy Premier League GameWeek 0. No games have been played in the English Premier League yet, thus various columns are initialized appropriately. There will be 37 additional datasets to be used for the 2017-2018 FPL season as each team in the English Premiership play 38 games to determine which team is the champion.

